# Use Case 1: Target Positions with Hedging

This notebook demonstrates:
1. Using target positions (shares, notional, weights)
2. Beta hedging for market neutrality
3. Sector hedging for sector neutrality
4. Comparing hedged vs unhedged performance

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from backtesting import Backtester, BacktestConfig, DataManager
from backtesting.utils import get_date_range
from notebooks.notebook_utils import setup_plotting_style, plot_results_summary

%matplotlib inline
setup_plotting_style()

## Load Data

In [ ]:
data_manager = DataManager(data_dir='../sample_data', use_float32=True)
prices = data_manager.load_prices()

# Get date range
start_date = prices.index[0]
end_date = prices.index[-1]

print(f"Data loaded: {len(prices.columns)} securities")
print(f"Date range: {start_date.date()} to {end_date.date()}")

## Example 1: Target Weights (No Hedging)

In [ ]:
# Load equal-weight targets
target_weights = pd.read_csv('../sample_data/target_weights.csv',
                              index_col=0, parse_dates=True)
target_weights = target_weights.loc[start_date:end_date]

targets_by_date = {
    date: target_weights.loc[date].to_dict()
    for date in target_weights.index
}

inputs = {'type': 'weights', 'targets': targets_by_date}

# Configure without hedging
config_no_hedge = BacktestConfig(
    initial_cash=10_000_000,
    max_adv_participation=0.05,
    enable_beta_hedge=False,
    enable_sector_hedge=False,
    risk_free_rate=0.02
)

# Run backtest
print("Running backtest WITHOUT hedging...")
backtester = Backtester(config_no_hedge, data_manager)
results_no_hedge = backtester.run(
    start_date=start_date,
    end_date=end_date,
    use_case=1,
    inputs=inputs,
    show_progress=True
)

## Example 2: Target Weights with Beta Hedging

In [ ]:
# Configure with beta hedging
config_beta_hedge = BacktestConfig(
    initial_cash=10_000_000,
    max_adv_participation=0.05,
    enable_beta_hedge=True,
    beta_hedge_instrument='SPY',
    target_beta=0.0,  # Market neutral
    enable_sector_hedge=False,
    risk_free_rate=0.02
)

# Run backtest
print("\nRunning backtest WITH beta hedging...")
backtester = Backtester(config_beta_hedge, data_manager)
results_beta_hedge = backtester.run(
    start_date=start_date,
    end_date=end_date,
    use_case=1,
    inputs=inputs,
    show_progress=True
)

## Example 3: Target Weights with Sector Hedging

In [ ]:
# Configure with sector hedging
config_sector_hedge = BacktestConfig(
    initial_cash=10_000_000,
    max_adv_participation=0.05,
    enable_beta_hedge=False,
    enable_sector_hedge=True,
    risk_free_rate=0.02
)

# Run backtest
print("\nRunning backtest WITH sector hedging...")
backtester = Backtester(config_sector_hedge, data_manager)
results_sector_hedge = backtester.run(
    start_date=start_date,
    end_date=end_date,
    use_case=1,
    inputs=inputs,
    show_progress=True
)

## Compare Results

In [ ]:
from notebooks.notebook_utils import compare_strategies

compare_strategies(
    [results_no_hedge, results_beta_hedge, results_sector_hedge],
    ['No Hedge', 'Beta Hedge', 'Sector Hedge'],
    metric='sharpe_ratio'
)

## Analyze Hedge Effectiveness

In [ ]:
# Get metrics for all three
metrics_no_hedge = results_no_hedge.calculate_metrics()
metrics_beta = results_beta_hedge.calculate_metrics()
metrics_sector = results_sector_hedge.calculate_metrics()

# Compare
comparison_df = pd.DataFrame({
    'No Hedge': metrics_no_hedge,
    'Beta Hedge': metrics_beta,
    'Sector Hedge': metrics_sector
})

print("\nMetrics Comparison:")
print(comparison_df.loc[['total_return', 'sharpe_ratio', 'max_drawdown', 
                         'volatility', 'win_rate']].round(4))

## Visualize Net Exposure Differences

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (results, title) in enumerate([
    (results_no_hedge, 'No Hedge'),
    (results_beta_hedge, 'Beta Hedge'),
    (results_sector_hedge, 'Sector Hedge')
]):
    df = results.to_dataframe()
    
    axes[idx].plot(df['date'], df['net_exposure'], label='Net', linewidth=2)
    axes[idx].axhline(y=0, color='red', linestyle='--', alpha=0.5)
    axes[idx].set_title(title)
    axes[idx].set_ylabel('Net Exposure ($)')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].legend()

plt.tight_layout()
plt.show()

print("\nNote: Beta hedge should keep net exposure closer to zero")

## Example 4: Target Notional

In [ ]:
# Create target notional (dollar amounts)
# Each security gets $100,000
target_notional_by_date = {}
for date in target_weights.index:
    target_notional_by_date[date] = {
        ticker: 100000.0 for ticker in target_weights.columns
    }

inputs_notional = {
    'type': 'notional',
    'targets': target_notional_by_date
}

# Run with notional targets
print("\nRunning with target notional...")
results_notional = backtester.run(
    start_date=start_date,
    end_date=end_date,
    use_case=1,
    inputs=inputs_notional,
    show_progress=True
)

results_notional.print_summary()

## Generate Reports

In [ ]:
# Generate reports for beta hedge case
results_beta_hedge.generate_full_report(
    output_dir='../output/use_case_1_beta_hedge',
    formats=['html', 'pdf', 'csv']
)

print("Reports saved to ../output/use_case_1_beta_hedge/")

## Summary

In this notebook, we explored Use Case 1:
1. ✓ Target positions with weights, notional, and shares
2. ✓ Beta hedging to achieve market neutrality
3. ✓ Sector hedging for sector neutrality
4. ✓ Compared hedged vs unhedged performance

Key Takeaways:
- Beta hedging reduces market exposure (net → 0)
- Sector hedging balances sector exposures
- Hedging can improve risk-adjusted returns
- Transaction costs increase with hedging trades